In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv("out/M1-C1-Udepl-120.csv")

In [22]:
rho = 19.01
d = 0.028

mass_1 = 0.2361
mass_2 = 0.2214

AVOGADRO = 6.02214076e+23
molar_mass = 238.0507847

In [6]:
mu_df = pd.read_csv("aux_data/mu_92.csv", index_col=0)

In [8]:
def get_mu_one(val, mu_df):
    if val in mu_df["photon_energy"]:
        return mu_df.loc[val, "total_attenuation_without_coherent_scatt"]
    lower = mu_df[mu_df["photon_energy"] < val].iloc[-1]
    upper = mu_df[mu_df["photon_energy"] > val].iloc[0]
    return lower["total_attenuation_without_coherent_scatt"] + ((val-lower["photon_energy"])/((upper-lower)["photon_energy"])) * ((upper-lower)["total_attenuation_without_coherent_scatt"]) 
    # mu_df.iloc[(mu_df[mu_df["photon_energy"] > val]['photon_energy']-val).abs().argsort()[:2]]

In [9]:
def get_mu_col(energy_col, mu_df):
    return energy_col.apply(lambda x: get_mu_one(x, mu_df))

In [10]:
mu = get_mu_col(df["Energy"], mu_df)

In [16]:
k = (mu * rho * d) / (1-np.exp(-mu * rho * d))

In [17]:
lam = np.log(2) / df["Half-life [s]"]

In [24]:
N = mass_1*AVOGADRO/molar_mass

In [26]:
t_irr = 30*60

In [28]:
irr_start = pd.to_datetime("1.12.2020 10:31:00")

In [30]:
irr_end = irr_start + pd.to_timedelta(t_irr, "s")

In [38]:
delta_t = pd.to_datetime(df["Acquisition Started"][0]) - irr_end

In [44]:
delta_t = delta_t.total_seconds()

In [47]:
df.columns.to_list()

['Energy',
 'E_tab',
 'Ig [%]',
 'Area',
 'Isotope',
 'sigm_E',
 'sigm_Ig',
 'FWHM',
 '%err',
 'Live Time',
 'Real Time',
 'Dead Time (rel)',
 'Ig',
 'IT',
 'Peak Analysis Performed on',
 'Report Generated On',
 'Channel',
 'Efficiency Calibration Used Done On',
 'Bkgnd',
 'Sample Taken On',
 'Pk',
 'Peak Analysis Report                    2.12.2020  10',
 'Energy Calibration Used Done On',
 'Cts/Sec',
 'Acquisition Started',
 'Identification Energy Tolerance',
 'eps',
 'Geometry',
 'Half-life [s]',
 'Stable',
 'Prod_mode_Photon reaction',
 'Prod_mode_Fast neutron activation',
 'Prod_mode_Naturally occurring',
 'sigm_Abundance',
 'sigm_Half-life [s]',
 'Prod_mode_Charged particle reaction',
 'Prod_mode_Thermal neutron activation',
 'Abundance',
 'Prod_mode_Fission product',
 'fiss_yield',
 'sigm_fiss_yield']

In [62]:
df["RR"] = ((df['Real Time'][0] / df["Live Time"][0]) * k * lam * df["Area"]) / (N * (1-np.exp(-lam*t_irr)) * np.exp(-lam * delta_t) * (1-np.exp(-lam * df["Real Time"][0])) * df["eps"] * df["Ig"])

In [71]:
df["RR"] = ((df['Real Time'][0] / df["Live Time"][0]) * k * lam * df["Area"]) #/ (N * (1-np.exp(-lam*t_irr)) * np.exp(-lam * delta_t) * (1-np.exp(-lam * df["Real Time"])) * df["eps"] * df["Ig"])

In [73]:
df["Area"]

0       786.0
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
5148      NaN
5149      NaN
5150      NaN
5151      NaN
5152      NaN
Name: Area, Length: 5153, dtype: float64

In [55]:
df["Real Time"]

0       321.4
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
5148      NaN
5149      NaN
5150      NaN
5151      NaN
5152      NaN
Name: Real Time, Length: 5153, dtype: float64